# Data Structures

> Changing a data structure in a slow program can work the same way an organ transplant does in a sick patient. Important classes of abstract data types such as containers, dictionaries, and priority queues, have many different but functionally equivalent data structures that implement them. Changing the data structure does not change the correctness of the program, since we presumably replace a correct implementation with a different correct implementation. However, the new implementation of the data type realizes different tradeoffs in the time to execute various operations, so the total performance can improve dramatically. Like a patient in need of a transplant, only one part might need to be replaced in order to fix the problem.

Steven S Skiena. The Algorithm Design Manual 

### Sequences and their Abstractions

#### What is a sequence?

Consider the notion of **Abstract Data Types**. 

The idea there is that one data type might be implemented in terms of another, or some underlying code, not even in python. 

As long as the interface and contract presented to the user is solid, we can change the implementation below. 

More on this later..

The **dunder methods** in python are used towards this purpose. 

In python a sequence is something that follows the sequence protocol. An example of this is a python list. 

This entails defining the `__len__` and `__getitem__` methods. 

In [1]:
alist=[1,2,3,4]
len(alist)#calls alist.__len__

4

In [2]:
alist[2]#calls alist.__getitem__(2)

3

Lists also support slicing. How does this work?

In [3]:
alist[2:4]

[3, 4]

To see this lets create a dummy sequence which shows us what happens. This sequence does not create any storage, it just implements the protocol

In [4]:
class DummySeq:
    
    def __len__(self):
        return 42
    
    def __getitem__(self, index):
        return index

In [5]:
d = DummySeq()
len(d)

42

In [6]:
d[5]

5

In [7]:
d[67:98]

slice(67, 98, None)

Slicing creates a `slice object` for us of the form `slice(start, stop, step)` and then python calls `seq.__getitem__(slice(start, stop, step))`.

What about two dimensional indexing, if we wanted to create a two dimensional structure?

In [8]:
d[67:98:2,1]

(slice(67, 98, 2), 1)

In [9]:
d[67:98:2,1:10]

(slice(67, 98, 2), slice(1, 10, None))

As sequence writers, our job is to interpret these in `__getitem__`

In [10]:
#taken from Fluent Python
import numbers, reprlib

class NotSoDummySeq:    
    def __init__(self, iterator):
        self._storage=list(iterator)
        
    def __repr__(self):
        components = reprlib.repr(self._storage)
        components = components[components.find('['):]
        return 'NotSoDummySeq({})'.format(components)
    
    def __len__(self):
        return len(self._storage)
    
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._storage[index])
        elif isinstance(index, numbers.Integral): 
            return self._storage[index]
        else:
            msg = '{cls.__name__} indices must be integers' 
            raise TypeError(msg.format(cls=cls))


In [11]:
d2 = NotSoDummySeq(range(10))
len(d2)

10

In [12]:
d2

NotSoDummySeq([0, 1, 2, 3, 4, 5, ...])

In [13]:
d[4]

4

In [14]:
d2[2:4]

NotSoDummySeq([2, 3])

In [15]:
d2[1,4]

TypeError: NotSoDummySeq indices must be integers

Just BTW, It might seem that slices are only useful in writing dunder methods like `__getitem__`, but this is not the case. Imagine you are writing code to parse a file:

In [16]:
mystring="""
Ralph 1 2
Midge 3 4
"""
name = slice(0,5)
weight =  slice(5,7)
height =  slice(7,9)
for line in mystring.split('\n')[1:-1]:
    print(line[name], line[weight].strip(), line[height].strip())

Ralph 1 2
Midge 3 4


### Linked Lists

Lets next think of how a list with a pointer to its next element might be constructed in python. We'll use this notion of a "linked list" to illustrate sequences.

#### Nested Pairs

![](http://wla.berkeley.edu/~cs61a/fa11/lectures/img/pair.png)

(from stanford cs61a, http://wla.berkeley.edu/~cs61a/fa11/lectures/objects.html#nested-pairs, this is the **box and pointer** notation.)

The diagram above can be represented in python as:

In [17]:
pair = (1,2)

But this representation lacks a certain power. What if we used this:

In [18]:
pair = (1, (2, None))

Which can generalize to something which looks like this:

![](http://wla.berkeley.edu/~cs61a/fa11/lectures/img/sequence.png)

In [19]:
linked_list = (1, (2, (3, (4, None))))

In [20]:
from IPython.display import HTML
HTML('<iframe width="1000" height="800" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=linked_list+%3D+(1,+(2,+(3,+(4,+None%29%29%29%29&origin=opt-frontend.js&cumulative=false&heapPrimitives=false&textReferences=false&py=2&rawInputLstJSON=%5B%5D&curInstr=1&codeDivWidth=300&codeDivHeight=200"> </iframe>')

#### Quick Linked List implementation

In [21]:
emptyll = None

def make_ll(first, rest):
        return (first, rest)
def first(ll):
        return ll[0]
def rest(ll):
        return ll[1]
myll = make_ll(1,make_ll(2, make_ll(3,emptyll)))
myll

(1, (2, (3, None)))

In [22]:
print(first(myll), "|", rest(myll), "|", first(rest(myll)))

1 | (2, (3, None)) | 2


Why might you want to use a linked list like the example above?

- you allocate memory only when you want to use it
- inserting a new element is cheaper than in a fixed size array
- gateway drug to other "pointer"ish and hierarchical structures.

### Linked Lists: A python implementation with the sequence protocol

In [23]:
from doctest import run_docstring_examples as dtest
import numbers
import reprlib
class LL:
    """
    >>> A = LL()  
    >>> A[0]
    Traceback (most recent call last):
        ...
    IndexError: trying to index an empty LL
    >>> A.insert_front(1)
    >>> A[0]
    1
    >>> A.insert_back(2)
    >>> A[1]
    2
    >>> A
    LL([1,...])
    >>> myll = LL.from_components([1,2])
    >>> myll[1]
    1
    >>> len(myll)
    2
    >>> myll[2]
    Traceback (most recent call last):
        ...
    IndexError: LL index out of range
    >>> myll[0:1]
    Traceback (most recent call last):
        ...
    TypeError: LL indices must be integers
    """
    @classmethod
    def from_components(cls, components):
        inst = cls(components[0])
        for c in components[1:]:
            inst.insert_front(c)
        return inst
        
    def __init__(self, head=None):
        if head is None:
            self._headNode = None
        else:
            self._headNode = [head, None]
            
    def insert_front(self, element):
        new_node = [element, None]
        new_node[1] = self._headNode
        self._headNode = new_node
        
    def insert_back(self, element):
        new_node = [element, None]
        curr_ptr = self._headNode
        while curr_ptr[1] is not None:
            curr_ptr = curr_ptr[1]
        curr_ptr[1]= new_node
        
    def __repr__(self):
        class_name = type(self).__name__
        if len(self)==0:
            components=""
        else:
            components = reprlib.repr(self[0])
        return '{}([{},...])'.format(class_name,components)


    def __len__(self):
        curr_ptr = self._headNode
        count = 0
        if curr_ptr==None:
            return 0
        while 1:
            count = count + 1
            if curr_ptr[1] is None:
                break
            curr_ptr = curr_ptr[1]
        return count    
    
    def __getitem__(self, index):
        class_name = type(self).__name__
        if isinstance(index, numbers.Integral): 
            curr_ptr = self._headNode
            if curr_ptr==None:
                msg = 'trying to index an empty {class_name}' 
                raise IndexError(msg.format(class_name=class_name))
            next_ptr = self._headNode[1]
            count = 0
            while 1:
                if index == count:
                    return curr_ptr[0]
                if curr_ptr[1] is None:
                    msg = '{class_name} index out of range' 
                    raise IndexError(msg.format(class_name=class_name))       
                count += 1
                curr_ptr = curr_ptr[1]
        else:
            msg = '{class_name} indices must be integers' 
            raise TypeError(msg.format(class_name=class_name))

In [24]:
from doctest import run_docstring_examples as dtest
dtest(LL, globals(), verbose = True)

Finding tests in NoName
Trying:
    A = LL()  
Expecting nothing
ok
Trying:
    A[0]
Expecting:
    Traceback (most recent call last):
        ...
    IndexError: trying to index an empty LL
ok
Trying:
    A.insert_front(1)
Expecting nothing
ok
Trying:
    A[0]
Expecting:
    1
ok
Trying:
    A.insert_back(2)
Expecting nothing
ok
Trying:
    A[1]
Expecting:
    2
ok
Trying:
    A
Expecting:
    LL([1,...])
ok
Trying:
    myll = LL.from_components([1,2])
Expecting nothing
ok
Trying:
    myll[1]
Expecting:
    1
ok
Trying:
    len(myll)
Expecting:
    2
ok
Trying:
    myll[2]
Expecting:
    Traceback (most recent call last):
        ...
    IndexError: LL index out of range
ok
Trying:
    myll[0:1]
Expecting:
    Traceback (most recent call last):
        ...
    TypeError: LL indices must be integers
ok


In [25]:
myll=LL.from_components([1,2,32,-4,5])
myll

LL([5,...])

In [26]:
min(myll), max(myll)

(-4, 32)

#### Linked List Complexity

From this implementation we can see some things:

- Insertion at the front of a linked list is O(1). Insertion at the back is O(n). 
- Searching a linked list is like linear search on an array: O(n)
- What about deletion? What about slices? See the homework :-)
- What about indexing? Its O(n) as you must follow the pointers.
- Setting a value (not implemented here) follows the indexing complexity.

The space complexity is O(n).

How does this contrast with a regular array?

### From pointers to Iterators

One can simply follow the `next` pointers to the next **POSITION** in a linked list. This suggests an abstraction of the **position** or pointer to an **iterator**, an abstraction which allows us to treat arrays and linked lists with an identical interface. 

The salient points of this abstraction are:

- the notion of a `next` abstracting away the actual gymnastics of where to go next in a storage system
- the notion of a `first` to a `last` that `next` takes us on a journey from and to respectively

- we already implemented the sequence protocol, but 
- now we suggest an additional abstraction that is more fundamental than the notion of a sequence: the **iterable**.

### Iterators and Iterables in python

Just as a sequence is something implementing `__getitem__` and `__len__`, an **Iterable** is something implementing `__iter__`. 

`__len__` is not needed and indeed may not make sense. 

The following example is taken from Fluent Python

In [27]:
import reprlib
class Sentence:
    def __init__(self, text): 
        self.text = text
        self.words = text.split()
        
    def __getitem__(self, index):
        return self.words[index] 
    
    def __len__(self):
        #completes sequence protocol, but not needed for iterable
        return len(self.words) 
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [28]:
#sequence'
a= Sentence("Mary had a little lamb whose fleece was white as snow.")
len(a), a[3], a

(11, 'little', Sentence('Mary had a l...hite as snow.'))

In [29]:
min(a), max(a)

('Mary', 'whose')

In [30]:
list(a)

['Mary',
 'had',
 'a',
 'little',
 'lamb',
 'whose',
 'fleece',
 'was',
 'white',
 'as',
 'snow.']

To iterate over an object x, python automatically calls `iter(x)`. An **iterable** is something which, when `iter` is called on it, returns an **iterator**.

(1) if `__iter__` is defined, calls that to implement an iterator.

(2) if not  `__getitem__` starting from index 0

(3) otherwise raise TypeError

Any Python sequence is iterable because they implement `__getitem__`. The standard sequences also implement `__iter__`; for future proofing you should too because  (2) might be deprecated in a future version of python.

This:

In [31]:
for i in a:
    print(i)

Mary
had
a
little
lamb
whose
fleece
was
white
as
snow.


is implemented something like this:

In [32]:
it = iter(a)
while True:
    try:
        nextval = next(it)
        print(nextval)
    except StopIteration:
        del it
        break

Mary
had
a
little
lamb
whose
fleece
was
white
as
snow.


`it` is an iterator. 

An iterator defines both `__iter__` and a `__next__` (the first one is only required to make sure an *iterator* IS an *iterable*). 

Calling `next` on an iterator will trigger the calling of `__next__`.

In [33]:
it=iter(a)#an iterator defines `__iter__` and can thus be used as an iterable
for i in it:
    print(i)

Mary
had
a
little
lamb
whose
fleece
was
white
as
snow.


In [34]:
it = iter(a)
next(it), next(it), next(it)

('Mary', 'had', 'a')

So now we can completely abstract away a sequence in favor an iterable (ie we dont need to support indexing anymore). From Fluent:

In [35]:
class SentenceIterator:
    def __init__(self, words): 
        self.words = words 
        self.index = 0
        
    def __next__(self): 
        try:
            word = self.words[self.index] 
        except IndexError:
            raise StopIteration() 
        self.index += 1
        return word 

    def __iter__(self):
        return self
    
class Sentence:#an iterable
    def __init__(self, text): 
        self.text = text
        self.words = text.split()
        
    def __iter__(self):
        return SentenceIterator(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [41]:
s2 = Sentence("While we could have implemented `__next__` in Sentence itself, making it an iterator, we will run into the problem of exhausting an iterator'.")

In [42]:
len(s2)

TypeError: object of type 'Sentence' has no len()

In [43]:
for i in s2:
    print(i)

While
we
could
have
implemented
`__next__`
in
Sentence
itself,
making
it
an
iterator,
we
will
run
into
the
problem
of
exhausting
an
iterator'.


In [44]:
s2it=iter(s2)
print(next(s2it))
s2it2=iter(s2)
next(s2it),next(s2it2)

While


('we', 'While')

While we could have implemented `__next__` in Sentence itself, making it an iterator, we will run into the problem of "exhausting an iterator". 

The iterator above keeps state in `self.index` and we must be able to start anew by creating a new instance if we want to re-iterate. Thus the `__iter__` in the iterable, simply returns the `SentenceIterator`.

In [45]:
min(s2), max(s2)

('Sentence', 'will')

Note that min and max will work even though we now DO NOT satisfy the sequence protocol, but rather the ITERABLE protocol, as its a pairwise comparison, which can be handled via iteration. The take home message is that in programming with these iterators, these generlization of pointers, we dont need either the length or indexing to work to implement many algorithms: we have abstracted these away.

## A brief addendum on trees

A tree is:

- a hierarchical data structure that has a bunch of items,
- each of which may have a value
- some of which may point to other such items, and some that dont (leaf nodes)
- each item is pointed to by exactly one other item, with the sole exception of the root.

Trees arise everywhere:

- in parsing of code
- evolutionary trees in biology
- language origin trees
- unix file system
- html tags on this page

Just like with lists, one can consider looking at a tree in two ways: a collection of nodes, or a tree with a root and multiple sub-tree's.

Once again, one can represent trees using the recursive data structures we used to represent linked lists (from cs61a):

![](http://wla.berkeley.edu/~cs61a/fa11/lectures/img/tree.png)



In [10]:
tree = ((1, 2), 3, 4)
tree - (((1, None), (2, None)), (3, None), (4, None))

You could also use a tree in which the nodes all themselves have data. This is often used to represent a binary tree.

In [12]:
class Tree:#from cs61a
    
    def __init__(self, data, left=None, right=None):
        self.entry = data
        self.left = left
        self.right = right
    def __repr__(self):
        args = repr(self.entry)
        if self.left or self.right:
            args += ', {0}, {1}'.format(repr(self.left), repr(self.right))
        return 'Tree({0})'.format(args)

Tree(1,Tree(2), Tree(3, Tree(4)))

Tree(1, Tree(2), Tree(3, Tree(4), None))

Once we do iteration in more detail, we'll talk about traversal mechanisms!